In [897]:
import pandas as pd

### Orders

In [898]:
# orders.csv
path = "C:/Users/ruben/local_repos/WBS/Data_Science/Section_3/Data_Cleaning_Storytelling_with_Pandas_and_Matplotlib/data/raw_data/orders.csv"
orders = pd.read_csv(path)

I'm looking for null values and data types.

In [899]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


Look's like 'total_paid' has 5 rows with null values and, 'created_date' Dtype needs to be changed.

In [900]:
orders["created_date"] = pd.to_datetime(orders["created_date"])

In [901]:
nan_count = orders.isna().sum()
nan_count_sum = nan_count.sum()

print(f"{nan_count_sum} missing values represents {((orders.total_paid.isna().sum() / orders.shape[0])*100).round(5)}% of the rows in our DataFrame")

5 missing values represents 0.0022% of the rows in our DataFrame


In [902]:
# create a boolean mask indicating which values are NaN
mask = orders.isna()

# use the mask to filter the dataframe
na_rows = orders[mask.any(axis=1)]

print(na_rows)

        order_id        created_date  total_paid    state
127701    427314 2017-11-20 18:54:39         NaN  Pending
132013    431655 2017-11-22 12:15:24         NaN  Pending
147316    447411 2017-11-27 10:32:37         NaN  Pending
148833    448966 2017-11-27 18:54:15         NaN  Pending
149434    449596 2017-11-27 21:52:08         NaN  Pending


Now we know which rows are bad, and details about that orders, some important one are 'created_date' and 'state'.

Let's figure it out for how long they are in 'pending' state.

In [903]:
first_order = orders["created_date"].min()
last_order = orders["created_date"].max()

In [904]:
total_period_time = orders["created_date"].max() - orders["created_date"].min()

In [905]:
print("This is the first created_date order: ")
first_order

This is the first created_date order: 


Timestamp('2017-01-01 00:07:19')

In [906]:
print("This is the last created_date order: ")
last_order

This is the last created_date order: 


Timestamp('2018-03-14 13:58:36')

In [907]:
# Total time in 'pending' state for the orders with 'NaN' total_paid value
from datetime import datetime

timestamps = na_rows['created_date']
timedeltas = []

for timestamp in timestamps:
    timedeltas.append(last_order - timestamp)

timedeltas

[Timedelta('113 days 19:03:57'),
 Timedelta('112 days 01:43:12'),
 Timedelta('107 days 03:25:59'),
 Timedelta('106 days 19:04:21'),
 Timedelta('106 days 16:06:28')]

We can drop them, without regrets, because they are in that state of 'pending' for an unusual long period of time, and also, the missing values represents only 0.0022% of the rows in our DataFrame.

In [908]:
orders = orders.dropna()

In [909]:
nan_count = orders.isna().sum()
nan_count = nan_count.sum()
nan_count

print(f"{nan_count} missing values represents {((orders.total_paid.isna().sum() / orders.shape[0])*100).round(5)}% of the rows in our DataFrame")

0 missing values represents 0.0% of the rows in our DataFrame


In [910]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  int64         
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 8.7+ MB


Everything look's nice and clean.

### Orderlines

In [911]:
# orderlines.csv
path = "C:/Users/ruben/local_repos/WBS/Data_Science/Section_3/Data_Cleaning_Storytelling_with_Pandas_and_Matplotlib/data/raw_data/orderlines.csv"
orderlines = pd.read_csv(path)

I'm looking for null values and data types.

In [912]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


Must be modified 'date' and 'unit_price'. 

In [913]:
orderlines["date"] = pd.to_datetime(orderlines["date"])

In [914]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


Now let's focus on 'unit_price'.

If you will run the line of code below, you will have this error: 'ValueError: Unable to parse string "1.137.99" at position 6'.

In [915]:
#orderlines["unit_price"] = pd.to_numeric(orderlines["unit_price"])

We need to create a function that keep only the last decimal point, which is the good one.

In [916]:
import re
def fix_two_dot(price):
  # Use a regex to find a string with two dots and capture the first and last parts of the string
  result = re.search(r'^(\d+)\.(\d+)\.(\d+)$', price)
  if result:
    # If a match is found, concatenate the captured groups and return the result
    return result.group(1) + result.group(2) + '.' + result.group(3)
  else:
    # If no match is found, return the original string
    return price

In [917]:
# Apply the custom function to the unit_price column
orderlines['unit_price'] = orderlines['unit_price'].apply(fix_two_dot)

Let's verify how many rows still have this 'two_dot' problem.

In [918]:
orderlines.loc[orderlines.unit_price.str.contains("\d+\.\d+\.\d+")]

,id,id_order,product_id,product_quantity,sku,unit_price,date


Look's like the problem was solved.

Let's verify the position 6 in dataframe, because of our last error: 'ValueError: Unable to parse string "1.137.99" at position 6'

In [919]:
orderlines.iloc[6]

id                              1119115
id_order                         299544
product_id                            0
product_quantity                      1
sku                             APP1582
unit_price                      1137.99
date                2017-01-01 01:17:21
Name: 6, dtype: object

Now must working nice and smooth 'pd.to_numeric' function applied to 'unit_price'.

In [920]:
orderlines["unit_price"] = pd.to_numeric(orderlines["unit_price"])

In [921]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 15.7+ MB


Everything look's nice and clean.

### Products

In [1313]:
# products.csv
path = "C:/Users/ruben/local_repos/WBS/Data_Science/Section_3/Data_Cleaning_Storytelling_with_Pandas_and_Matplotlib/data/raw_data/products.csv"
products = pd.read_csv(path)

I'm looking for null values and data types.

In [1314]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


Look's like we only need to change the Dtype of 'price' and 'promo_price'.

Let's have a closer look.

In [1315]:

products.head(20)

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364
5,APP0073,Apple Composite AV Cable iPhone and iPod white,IPhone and iPod AV Cable Dock to Composite Video.,45,420.003,0,1230
6,KIN0008,Mac Memory Kingston 1GB 667MHz DDR2 SO-DIMM,1GB RAM Mac mini and iMac (2006/07) MacBook Pr...,18.99,146.471,0,1364
7,KIN0009,Mac Memory Kingston 2GB 800MHz DDR2 SO-DIMM,2GB RAM iMac with Intel Core 2 Duo (Penryn).,36.99,274.694,0,1364
8,KIN0001-2,Mac memory Kingston 4GB (2x2GB) 667MHz DDR2 SO...,RAM 4GB (2x2GB) Mac mini and iMac (2006/07) Ma...,74,669.904,0,1364
9,APP0100,Apple Adapter Mini Display Port to VGA,Adapter Mini Display Port to VGA MacBook and M...,35,330.003,0,1325


For the first glanz look's like 'promo_price' need to be fixed.

When 'promo_price' has more characters before his decimal point, compared with 'price', it seems to be only a problem to move the decimal point one position to the left on the 'promo_price' column.

#### 'promo_price' column

First let try to solve two dot problem. In case exist, it will be solved.

In [1316]:
products = products.astype({"promo_price": 'str', "price": 'str'})

In [1317]:
# Apply the custom function to the promo_price column
products['promo_price'] = products['promo_price'].apply(fix_two_dot)
products['price'] = products['price'].apply(fix_two_dot)

Now we can try to to see if we have some 'NaN' values in our dataframe.

In [1318]:
#products['promo_price'] = pd.to_numeric(products['promo_price'])
#products['price'] = pd.to_numeric(products['price'])

If we try tu run the code above we'll have this error:

'ValueError: Unable to parse string "nan" at position 34'

In [1319]:
nan_price = products['price'].loc[products['price'] == 'nan'].count()
nan_price

46

In [1320]:
print(f"The column price has in total {nan_price} 'nan' values. This is {round(((nan_price / products.shape[0]) * 100), 2)}% of the rows of the DataFrame")

The column price has in total 46 'nan' values. This is 0.24% of the rows of the DataFrame


0.24% is not too much and more important, the price column will be important to understanding discounts, so I'd like it to be very trustworthy as we are basing business decisions on it. Therefore, we'll delete these rows.

In [1321]:
i = products[products['price'] == 'nan'].index
products = products.drop(i)

Let's try to move the decimal dot one position to the left

In [1322]:
def modify_promo_price(row):
    price = row['price']
    promo_price = row['promo_price']
    if len(promo_price.split('.')[0]) > len(price.split('.')[0]):
        # Extract the integer and decimal parts of the promo_price
        int_part, dec_part = re.match(r'(\d+)\.(\d+)', promo_price).groups()
        # Reassemble the promo_price with the decimal point moved one place to the left
        magic = len(int_part) - 1
        promo_price = '{}.{}{}'.format(int_part[:magic], int_part[magic:], dec_part[0:])
    return promo_price

In [1323]:
# Apply the custom function to the promo_price column
products['promo_price'] = products.apply(modify_promo_price, axis=1)

In [1324]:
products['promo_price'] = pd.to_numeric(products['promo_price'])
products['price'] = pd.to_numeric(products['price'])


In [1325]:
products.head(20)

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,49.9899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,58.9996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.00,56.9898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.00,22.9997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.9900,1,1364
5,APP0073,Apple Composite AV Cable iPhone and iPod white,IPhone and iPod AV Cable Dock to Composite Video.,45.00,42.0003,0,1230
6,KIN0008,Mac Memory Kingston 1GB 667MHz DDR2 SO-DIMM,1GB RAM Mac mini and iMac (2006/07) MacBook Pr...,18.99,14.6471,0,1364
7,KIN0009,Mac Memory Kingston 2GB 800MHz DDR2 SO-DIMM,2GB RAM iMac with Intel Core 2 Duo (Penryn).,36.99,27.4694,0,1364
8,KIN0001-2,Mac memory Kingston 4GB (2x2GB) 667MHz DDR2 SO...,RAM 4GB (2x2GB) Mac mini and iMac (2006/07) Ma...,74.00,66.9904,0,1364
9,APP0100,Apple Adapter Mini Display Port to VGA,Adapter Mini Display Port to VGA MacBook and M...,35.00,33.0003,0,1325


In [1326]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19280 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          19280 non-null  object 
 1   name         19280 non-null  object 
 2   desc         19273 non-null  object 
 3   price        19280 non-null  float64
 4   promo_price  19280 non-null  float64
 5   in_stock     19280 non-null  int64  
 6   type         19230 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 1.2+ MB


Problem solved.

### My curiosity section.

I want a function that verify each row, if the values of 'promo_price' are grater than 'price' and return True or False for respectively situation.

In [1327]:
def check_price(row):
  # Convert the values in the price and promo_price columns to floats
  price = float(row['price'])
  promo_price = float(row['promo_price'])

  # Check if the promo_price is greater than the price
  if promo_price > price:
    # If it is, return True
    return True
  else:
    # If it is not, return False
    return False

In [1328]:
# Apply the custom function to each row of the dataframe
products['is_promo_price_greater'] = products.apply(check_price, axis=1)

Verify how much are in that case and how much are not.

In [1329]:
products['is_promo_price_greater'].value_counts()

False    18132
True      1148
Name: is_promo_price_greater, dtype: int64

In [1330]:
# the True rate of dataframe
true_count = products['price'][products['is_promo_price_greater'] == True].count() 
total_count = products.shape[0]
true_rate = round(true_count / total_count *100, 2) 

print(f"{true_rate}% of the rows in products.csv, 'promo_price' is grater than 'price'")

5.95% of the rows in products.csv, 'promo_price' is grater than 'price'


Hmm, I don't like it. Look's enough weird to make me want to investigate more.

In [1331]:
products.loc[products['is_promo_price_greater'] == True].head(10)

,sku,name,desc,price,promo_price,in_stock,type,is_promo_price_greater
15,MOS0021,Clearguard Moshi MacBook Pro and Air,Keyboard Protector MacBook Pro 13-inch Retina ...,24.95,24.9889,0,13835403,True
19,APP0234,Apple Dock Connector to VGA,Dock Connector to VGA IOS.,35.00,35.0005,0,13955395,True
39,JMO0026,Just Mobile Lazy Couch Support Mac and iPad,Mac and iPad small lift stand.,19.95,19.9892,0,8696,True
52,BLL0008,BlueLounge CableDrop Pack 6 Black,Cable management Mac iPhone iPad and iPod.,11.99,11.9911,0,12285400,True
55,BLL0012,Milo BlueLounge support iPhone and iPod white,Desktop support iPhone and iPod.,14.99,14.9919,0,1216,True
68,MAK0009,Maclocks theft case iPad 2 3 and 4 transparent,Case antitheft iPad 2 3 and 4 polycarbonate su...,79.99,79.9931,0,12635403,True
69,MAK0010,Maclocks theft case iPad 2 3 and 4 black,Case antitheft iPad 2 3 and 4 polycarbonate su...,79.99,79.9931,0,12635403,True
70,MAK0012,"Maclocks theft cover Macbook Air 13 """,Theft Case MacBook Air 13-inch polycarbonate w...,79.99,79.9931,0,13835403,True
74,SYN0051,Sliding Rail kit synology 2U sliding rails,Shelf rail kit Synology RackStation.,102.49,929.8850,0,1404,True
76,WAC0045,Wacom Intuos Airbrush stylus 4/5,stylus for pen tablets Cintiq 21UX Intuos 4/5 ...,109.90,999.9440,0,1229,True


Look's like for some of them is only a decimal dot problem.

Let's try to solve it and see what we have after that.

I want to create a function that calculates with how much is 'promo_price' grater than 'price'.

In [1332]:
def promo_greater_with(row):
    if row['promo_price'] > row['price']:
        return row['promo_price'] - row['price']
    else:
        return 0

In [1333]:
products['promo_greater_with'] = products.apply(promo_greater_with, axis=1)

In [1334]:
products.nlargest(100, 'promo_greater_with')

,sku,name,desc,price,promo_price,in_stock,type,is_promo_price_greater,promo_greater_with
18431,APP2662,"Apple iMac Pro 27 ""10-core Intel Xeon 3GHz W |...",Pro iMac 27 inch screen Retina 5K and Intel Xe...,10539.00,99071.134,0,118692158,True,88532.134
18426,APP2657,"Apple iMac Pro 27 ""8-core Intel Xeon W 32GHz |...",Pro iMac 27 inch screen Retina 5K and Intel Xe...,10539.00,99070.045,0,118692158,True,88531.045
18449,APP2680,"Apple iMac Pro 27 ""18-core Intel Xeon W 23GHz ...",Pro iMac 27 inch screen Retina 5K and Intel Xe...,10299.00,96810.043,0,118692158,True,86511.043
18451,APP2682,"Apple iMac Pro 27 ""10-core Intel Xeon 3GHz W |...",Pro iMac 27 inch screen Retina 5K and Intel Xe...,10299.00,96810.043,0,118692158,True,86511.043
18475,APP2706,"Apple iMac Pro 27 ""14-core Intel Xeon W 25GHz ...",Pro iMac 27 inch screen Retina 5K and Intel Xe...,10299.00,96810.043,0,118692158,True,86511.043
...,...,...,...,...,...,...,...,...,...
11180,PAC1646,QNAP TS-253A | 8GB RAM | 16TB (2x8TB) WD Red,NAS 8GB memory RAM + 16TB (2x8TB) Seagate Hard...,1217.99,9281.789,0,12175397,True,8063.799
11181,PAC1646,QNAP TS-253A | 8GB RAM | 16TB (2x8TB) WD Red,NAS 8GB memory RAM + 16TB (2x8TB) Seagate Hard...,1217.99,9281.789,0,12175397,True,8063.799
16941,APP1575-A,"Open - Apple MacBook Air 13 ""i5 16 Ghz | 8GB R...",Laptop MacBook Air 13 inch i5 128GB RAM 8GB Fl...,1099.00,9146.874,0,1298,True,8047.874
11026,PAC1301,Pack QNAP TS-451 + | 2GB RAM | Seagate 16TB Ir...,Pack QNAP TS-251 + with 2GB RAM + 16TB (4x4TB)...,1198.95,9223.685,0,12175397,True,8024.735


Look's like for some of them is still a decimal problem.

I want to order them, and reach more insights.

In [1335]:
sorted_products = products.copy()
sorted_products.sort_values(by='promo_greater_with', inplace=True)

In [1336]:
sorted_products.loc[sorted_products['promo_greater_with'] > 1000].head(50)

,sku,name,desc,price,promo_price,in_stock,type,is_promo_price_greater,promo_greater_with
17992,PAC2281,"Second hand - Apple iMac 20 ""Core 2 Duo 226GHz...",Refurbished iMac 20 inch Core 2 Duo | 2GB RAM ...,1499.00,2555.945,0,"5,43E+15",True,1056.945
17668,PAC2196,"Second hand - Apple iMac 20 ""Core 2 Duo 226GHz...",Refurbished iMac 20 inch Core 2 Duo | 2GB RAM ...,1199.00,2555.945,0,"5,43E+15",True,1356.945
17667,PAC2195,"Second hand - Apple iMac 20 ""Core 2 Duo 226GHz...",Refurbished iMac 20 inch Core 2 Duo | 2GB RAM ...,1199.00,2555.945,0,"5,43E+15",True,1356.945
17665,PAC2194,"Second hand - Apple iMac 20 ""Core 2 Duo 226GHz...",Refurbished iMac 20 inch Core 2 Duo | 2GB RAM ...,1199.00,2555.945,0,"5,43E+15",True,1356.945
17705,PAC2200,"Second hand - Apple iMac 20 ""Core 2 Duo 226GHz...",Refurbished iMac 20 inch Core 2 Duo | 2GB RAM ...,1199.00,2555.945,0,"5,43E+15",True,1356.945
17991,PAC2282,"Second hand - Apple iMac 20 ""Core 2 Duo 226GHz...",Refurbished iMac 20 inch Core 2 Duo | 2GB RAM ...,1199.00,2555.945,0,"5,43E+15",True,1356.945
17704,PAC2198,"Second hand - Apple iMac 20 ""Core 2 Duo 226GHz...",Refurbished iMac 20 inch Core 2 Duo | 2GB RAM ...,1199.00,2755.944,0,"5,43E+15",True,1556.944
16041,PAC2062,"Second hand - Apple iMac 20 ""Core 2 Duo 266GHz...",IMac used 20 inch Core 2 Duo 266GHz | 4GB RAM ...,1499.00,3355.948,0,51882158,True,1856.948
16039,PAC2063,"Second hand - Apple iMac 20 ""Core 2 Duo 266GHz...",Refurbished iMac 20 inch Core 2 Duo 266GHz | 4...,1499.00,3455.949,0,51882158,True,1956.949
14996,PAC1921,"Second hand - Apple iMac 20 ""Core 2 Duo 24GHz ...",IMac used 20 inch Core 2 Duo 24GHz | 3GB RAM |...,1199.00,3255.844,0,1282,True,2056.844


Till now confirm that.

In [1337]:
sorted_products.loc[sorted_products['promo_greater_with'] > 500].head(50)

,sku,name,desc,price,promo_price,in_stock,type,is_promo_price_greater,promo_greater_with
16958,NTE0019-A,Open - NewerTech Voyager Q HDD Dock FW800 / 40...,Dock disc 25 inch and 35 inch FW800 / 400 / US...,108.99,609.294,0,1298,True,500.304
15132,PLA0014-A,Open - Plantronics Backbeat Fit Green Wireless...,Bluetooth sports headset with microphone and w...,129.99,643.593,0,1298,True,513.603
433,NTE0025,NewerTech Voyager Q HDD Dock FW800 / 400 / USB...,Dock disc 25 inch and 35 inch FW800 / 400 / US...,108.99,626.901,0,12995397,True,517.911
18718,MAT0003-A,Open - Matias Bluetooth keyboard and numeric S...,Refurbished Bluetooth keyboard aluminum numeri...,119.99,651.044,0,5401,True,531.054
2322,MOP0065,Mophie Juice Pack Plus Battery Case (3300 mAh)...,Case 3300 mAh battery and LED indicator for iP...,129.99,669.856,0,"5,49E+11",True,539.866
14450,MOP0058-A,(Open) Mophie Juice Pack Plus Battery Case (33...,Case 3300 mAh battery and LED indicator for iP...,129.99,670.877,0,1298,True,540.887
1056,MOP0056,Mophie Space Pack Battery Case (1700mAh) and S...,Housing with battery and 16GB external storage...,149.99,699.864,0,"5,49E+11",True,549.874
13121,TPL0034,TP-Link TL-PA8010P Kit 1 Port Gigabit Powerlin...,amplifier with fast speed internet plug kit in...,119.79,669.892,0,1334,True,550.102
13681,LIF0095-A,Open - Lifeproof nüüd Waterproof Case iPad Min...,Waterproof protective cover and extreme condit...,109.99,671.868,0,1298,True,561.878
15005,WIT0017-A,Open - Withings Activite POP Black Clock,Smartwatch that records sleep and movement for...,149.90,717.738,0,1298,True,567.838


Still confirm that.

In [1338]:
sorted_products.loc[sorted_products['promo_greater_with'] > 250].head(50)

,sku,name,desc,price,promo_price,in_stock,type,is_promo_price_greater,promo_greater_with
2345,ADN0016-A,Open - Adonit Jot Touch with PixelPoint pointe...,Special precision pointer to draw fine tip,119.000,380.832,0,1298,True,261.832
11626,JAW0039,Jawbone UP2 Activity Monitor Plata,Bluetooth monitor activity and sleep with pers...,119.900,399.905,0,11905404,True,280.005
18989,PAC2507,Keyboard Replacement numerical Wireless iMac,Keyboard replacement service at the time of pu...,149.000,499.899,1,13855401,True,350.899
12628,LIF0050-A,(Open) Lifeproof nüüd Waterproof Case iPad Min...,Water resistant protective cover for iPad Mini...,109.990,471.427,0,1298,True,361.437
3036,PQI0008,IConnect PQI 64GB USB 3.0 Silver Lightning,PenDrive 64GB USB 3.0 and Lightning for iPhone...,139.900,502.803,0,42945397,True,362.903
2821,OWC0069-A,Open - OWC Envoy Pro USB 3.0 Case for MacBook ...,Portable USB 3.0 External SSD box Retina MacBo...,118.990,485.655,0,1298,True,366.665
1863,RUT0009,Orbit runtastic fitness monitor activity and s...,Bracelet sport and activity control sleep for ...,119.990,499.899,0,11905404,True,379.909
1114,ADN0022,Adonit Jot Touch with PixelPoint pointer iPad ...,Pointer with fine tip and precision IPAD,119.000,499.899,0,1229,True,380.899
15530,LEX0028-A,Open - Lexar JumpDrive Lightning USB 3.0 C20i ...,Lightning to USB external memory for iPad and ...,108.900,501.182,0,1298,True,392.282
84,IKM0013,IK Multimedia iRig MIX DJ Mixer iPhone iPad an...,Dj mixer iPhone iPad and iPod.,121.990,519.937,0,11905404,True,397.947


Still.

In [1339]:
sorted_products.loc[sorted_products['promo_greater_with'] > 50].head(50)

,sku,name,desc,price,promo_price,in_stock,type,is_promo_price_greater,promo_greater_with
2883,APP1096,Apple iPod Touch 64GB Blue,New 6th generation iPod Touch 64GB with 8 mega...,292.810,342.8112,0,11821715,True,50.0012
2882,APP1093,Apple iPod Touch 64GB Rosa,New 6th generation iPod Touch 64GB with 8 mega...,292.810,342.8112,0,11821715,True,50.0012
2881,APP1090,Apple iPod Touch 64GB Space Gray,New 6th generation iPod Touch 64GB with 8 mega...,292.810,342.8112,0,11821715,True,50.0012
2879,APP1084,Apple iPod Touch 64GB Silver,New 6th generation iPod Touch 64GB with 8 mega...,292.810,342.8112,0,11821715,True,50.0012
2880,APP1087,Apple iPod Touch 64GB Gold,New 6th generation iPod Touch 64GB with 8 mega...,292.810,342.8112,0,11821715,True,50.0012
11189,PAC1655,QNAP TS-253A | 16GB RAM | 6TB (2x3TB) WD Red,NAS with 16GB of RAM + memory 6TB (2x3TB) Netw...,7860.782,7911.7910,1,12175397,True,51.0090
11188,PAC1655,QNAP TS-253A | 16GB RAM | 6TB (2x3TB) WD Red,NAS with 16GB of RAM + memory 6TB (2x3TB) Netw...,7860.782,7911.7910,1,12175397,True,51.0090
11339,KIN0153,Kingston DataTraveler G2 USB 3.0 8Gb SE9,8GB USB 3.0 flash drive light and small.,15.990,67.7960,0,57445397,True,51.8060
10754,NTE0022-A,Open - NewerTech Mini DVI to VGA Adapter,Mini DVI to VGA.,18.990,71.2630,0,1298,True,52.2730
13550,GRT0428,Griffin Reveal iPhone Case 8/7 Transparent,thin polycarbonate case with transparent back ...,16.990,69.9020,0,11865403,True,52.9120


My conclusion is:
- for the difference between 'promo_price' and 'price', grater than 500 is only a decimal problem.
- for the difference smaller than 500, also most of them is only a decimal problem, but we have some products with 'price' with more than 4 figures, that the dot logic cannot work.


In [1340]:
(products['promo_greater_with'] > 18.9012).value_counts()

False    18535
True       745
Name: promo_greater_with, dtype: int64

#### Sample test function

In [1341]:
products_test = pd.DataFrame({'price': ['10.01', '11.01', '12.01'], 
                              'promo_price': ['100.2', '110.2', '120.2'],
                              'promo_greater_with': [90.19,99.19,108.19]})

In [1342]:
products_test

,price,promo_price,promo_greater_with
0,10.01,100.2,90.19
1,11.01,110.2,99.19
2,12.01,120.2,108.19


In [1343]:
def modify_promo_price_2_sample(row):
    price = row['price']
    promo_price = row['promo_price']
    promo_greater_with = row['promo_greater_with']
    if promo_greater_with > 90:
        # Extract the integer and decimal parts of the promo_price
        int_part, dec_part = re.match(r'(\d+)\.(\d+)', promo_price).groups()
        # Reassemble the promo_price with the decimal point moved one place to the left
        if len(int_part) < 4:
            magic = len(int_part) - 1
            promo_price = '{}.{}{}'.format(int_part[:magic], int_part[magic:], dec_part[0:])
    return promo_price

In [1344]:
products_test['promo_price'] = products_test.apply(modify_promo_price_2_sample, axis=1)

In [1345]:
products_test

,price,promo_price,promo_greater_with
0,10.01,10.02,90.19
1,11.01,11.02,99.19
2,12.01,12.02,108.19


#### Modify promo price again

##### First let's do the job for everything grater than 500

In [1346]:
def modify_promo_price_with_grather_condition(row):
    price = row['price']
    promo_price = row['promo_price']
    promo_greater_with = row['promo_greater_with']
    if promo_greater_with > 500:
        # Extract the integer and decimal parts of the promo_price
        int_part, dec_part = re.match(r'(\d+)\.(\d+)', promo_price).groups()
        # Reassemble the promo_price with the decimal point moved one place to the left
        magic = len(int_part) - 1
        promo_price = '{}.{}{}'.format(int_part[:magic], int_part[magic:], dec_part[0:])
    return promo_price

In [1347]:
products = products.astype({"promo_price": 'str', "price": 'str'})

In [1348]:
products['promo_price'] = products.apply(modify_promo_price_with_grather_condition, axis=1)

In [1349]:
products['promo_price'] = pd.to_numeric(products['promo_price'])
products['price'] = pd.to_numeric(products['price'])

#### Verify what we have now.

In [1350]:
products['promo_greater_with'] = products.apply(promo_greater_with, axis=1)

In [1351]:
# Apply the custom function to each row of the dataframe
products['is_promo_price_greater'] = products.apply(check_price, axis=1)

In [1352]:
products['is_promo_price_greater'].value_counts()

False    18635
True       645
Name: is_promo_price_greater, dtype: int64

In [1354]:
sorted_products = products.copy()
sorted_products.sort_values(by='promo_greater_with', inplace=True)

In [ ]:
sorted_products.loc[sorted_products['promo_greater_with'] > 50].head(10)

##### Let's do the job for everything grater than 50, but excepting the prices with 4 figures

In [1358]:
def modify_promo_price_with_grather_condition2(row):
    price = row['price']
    promo_price = row['promo_price']
    promo_greater_with = row['promo_greater_with']
    if promo_greater_with > 50:
        # Extract the integer and decimal parts of the promo_price
        int_part, dec_part = re.match(r'(\d+)\.(\d+)', promo_price).groups()
        # Reassemble the promo_price with the decimal point moved one place to the left
        if len(int_part) < 4:
            magic = len(int_part) - 1
            promo_price = '{}.{}{}'.format(int_part[:magic], int_part[magic:], dec_part[0:])
    return promo_price

In [1359]:
products = products.astype({"promo_price": 'str', "price": 'str'})

In [1360]:
products['promo_price'] = products.apply(modify_promo_price_with_grather_condition2, axis=1)

In [1361]:
products['promo_price'] = pd.to_numeric(products['promo_price'])
products['price'] = pd.to_numeric(products['price'])

#### Verify what we got

In [1362]:
products['promo_greater_with'] = products.apply(promo_greater_with, axis=1)
# Apply the custom function to each row of the dataframe
products['is_promo_price_greater'] = products.apply(check_price, axis=1)
products['is_promo_price_greater'].value_counts()

False    18817
True       463
Name: is_promo_price_greater, dtype: int64

In [1363]:
sorted_products = products.copy()
sorted_products.sort_values(by='promo_greater_with', inplace=True)

In [1370]:
sorted_products.loc[sorted_products['promo_greater_with'] > 50]

,sku,name,desc,price,promo_price,in_stock,type,is_promo_price_greater,promo_greater_with
11188,PAC1655,QNAP TS-253A | 16GB RAM | 6TB (2x3TB) WD Red,NAS with 16GB of RAM + memory 6TB (2x3TB) Netw...,7860.782,7911.7910,1,12175397,True,51.0090
11189,PAC1655,QNAP TS-253A | 16GB RAM | 6TB (2x3TB) WD Red,NAS with 16GB of RAM + memory 6TB (2x3TB) Netw...,7860.782,7911.7910,1,12175397,True,51.0090
12854,SAN0139,"SanDisk SSD 480GB Plus 25 ""SATA 6Gb / s",Hard SSD 480GB 25 inches,1499.892,1565.8370,0,12215397,True,65.9450
11258,PAC1675,QNAP TS-453A | 16GB | 12TB (4x3TB) WD Red,QNAP NAS TS-453A with 16GB of RAM memory + 12T...,11892.782,11963.6810,1,12175397,True,70.8990
12880,PAC2037,Synology DS216 + II | 8GB RAM,NAS with 4K transcoding and direct copy button...,5079.696,5151.7930,0,12175397,True,72.0970
14719,QNA0190,QNAP TVS-871U-RP NAS Server | Core i5 | 8GB RA...,NAS system format rack 8 bays and 8GB RAM,2418.790,2538.9902,0,12175397,True,120.2002
12868,PAC2025,Synology DS216 + II | 2GB RAM,NAS with 4K transcoding and direct copy button...,3739.696,3861.7880,0,12175397,True,122.0920
15111,EIZ0022,"CS2730 Monitor Eizo ColorEdge 27 ""QHD AdobeRGB...",Professional monitor for color management Adob...,1034.990,1209.9879,1,1296,True,174.9979
14716,QNA0187,QNAP TVS-1271U-RP NAS server with redundant po...,Expansion unit rack format with 12 bays and 32 GB,3386.790,3567.9899,0,12175397,True,181.1999
14717,QNA0188,QNAP TVS-1271U-RP NAS server with redundant po...,Expansion unit rack bays 12 and format memory ...,2781.790,2962.9899,0,12175397,True,181.1999


Look's good, I will let it like this.

### Save the clean data.

In [1374]:
# path for clean data
path_clean_data = "C:/Users/ruben/local_repos/WBS/Data_Science/Section_3/Data_Cleaning_Storytelling_with_Pandas_and_Matplotlib/data/clean_data/"

In [1375]:
orders.to_csv(path_clean_data + "orders_cl.csv", index=False)

In [1376]:
orderlines.to_csv(path_clean_data + "orderlines_cl.csv", index=False)

In [1377]:
products.to_csv(path_clean_data + "products_cl.csv", index=False)